In [7]:
!pip install tensorflow


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install pandas scikit-learn torch h5py


### Install dependencies


In [9]:
!pip install -q scikit-learn pandas matplotlib torch

import os
import pandas as pd
import numpy as np
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt




In [10]:
def load_csvs(folder, use_anomaly=True):
    files = sorted(glob(folder + '/*.csv'))
    all_rows = []
    for file in files:
        df = pd.read_csv(file, sep=';')
        df.drop(['datetime', 'changepoint'], axis=1, inplace=True, errors='ignore')
        if not use_anomaly:
            df['anomaly'] = 0
        all_rows.append(df)
    return pd.concat(all_rows, ignore_index=True)

base = '/content/drive/MyDrive/data'

df_clean = load_csvs(os.path.join(base, 'anomaly-free'), use_anomaly=False)
df_valve1 = load_csvs(os.path.join(base, 'valve1'))
df_valve2 = load_csvs(os.path.join(base, 'valve2'))
df_other = load_csvs(os.path.join(base, 'other'))

df = pd.concat([df_clean, df_valve1, df_valve2, df_other], ignore_index=True)
df = df.dropna()
df.head(5)

,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure,Temperature,Thermocouple,Voltage,Volume Flow RateRMS,anomaly
0,0.202394,0.275154,2.16975,0.382638,90.6454,26.8508,238.852,122.664,0.0
1,0.203153,0.277857,2.07999,-0.273216,90.7978,26.8639,227.943,122.338,0.0
2,0.202054,0.275790,2.52577,0.382638,90.7730,26.8603,223.486,121.338,0.0
3,0.203595,0.278101,2.49742,0.054711,90.8424,26.8616,244.904,121.664,0.0
4,0.201889,0.276363,2.29194,0.710565,90.6664,26.8603,239.196,122.000,0.0


In [11]:
#2. Preprocessing and Windowing

features = df.drop('anomaly', axis=1).values
labels = df['anomaly'].values

scaler = MinMaxScaler()
features = scaler.fit_transform(features)

def make_windows(data, labels, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(int(np.any(labels[i:i+window_size])))
    return np.array(X), np.array(y)

# window size
window_size = 90
X, y = make_windows(features, labels, window_size)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)


In [12]:
#3. Define Model (under 1000 params)
class SmallCNN(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(input_size, 8, kernel_size=3),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Linear(8, 1)

    def forward(self, x):
        x = self.conv(x).squeeze(-1)
        return torch.sigmoid(self.fc(x))

model = SmallCNN(X.shape[2])


In [13]:
#4. Training
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

X_train_t = torch.tensor(X_train, dtype=torch.float32).transpose(1, 2)
y_train_t = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_t = torch.tensor(X_test, dtype=torch.float32).transpose(1, 2)
y_test_t = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

train_ds = TensorDataset(X_train_t, y_train_t)
test_ds = TensorDataset(X_test_t, y_test_t)
train_dl = DataLoader(train_ds, batch_size=128, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=128)

loss_fn = nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.001)


In [14]:
train_losses = []
for epoch in range(100):
    model.train()
    batch_losses = []
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = loss_fn(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_losses.append(loss.item())
    train_losses.append(np.mean(batch_losses))


In [15]:
#5. Evaluation and Save
model.eval()
with torch.no_grad():
    preds = model(X_test_t.to(device)).cpu().numpy()
    preds = (preds > 0.5).astype(int)
    acc = (preds == y_test.reshape(-1, 1)).mean()
    print(f'Accuracy: {acc:.4f}')

# Save model
torch.save(model.state_dict(), 'model_ws413.h5')


Accuracy: 0.8458
